## Export data to SpaceTX format

First create the subclasses of `FetchedTile` and `TileFetcher` required and then export the data into SpaceTX

In [ ]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
#        print(np.shape(self.tile_data()))
        return {Axes.Y: 7962, Axes.X: 3356}
#         return {Axes.Y: 1000, Axes.X: 500}
#         return {Axes.Y: 1024, Axes.X: 1024}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
#         crp = img[2000:6000, 1000:1800]
        crp = img[3000:4000, 1300:1800]
#         crp = img[2950:3150, 1330:1730]
#         crp = img[1024:2048, 0:1024]
        return crp
    
    def tile_data(self) -> np.ndarray:
#        return self.crop(imread(self.file_path))
       return (imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C{ch_label+1}_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C0_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 4,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"dapi": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "GFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "Cy5"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 3, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "iRFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

In [ ]:
format_data("/home/nomi/Desktop/starfish/raw_data/2019-06-29_Justus_section3_round1_2x10_1",
           "/home/nomi/Desktop/starfish/experiment")

## Load the experiment and visualize the codebook
Possibly necessary to copy the codebook information from `codebook_backup.json`.

In [ ]:
%load_ext autoreload
%autoreload 2
from starfish import Experiment
from six.moves import urllib
from slicedimage.backends import CachingBackend, DiskBackend, HttpBackend, SIZE_LIMIT

baseurl: str="/home/nomi/Desktop/starfish/experiment/experiment.json"
experiment = Experiment.from_json(baseurl)
experiment.codebook

## Extract FOV/primary images, and project onto the Z axis
The slider in the viewer should now only have as many options as there are channels.

In [ ]:
fov: starfish.ImageStack = experiment["fov_000"].max_proj(Axes.ZPLANE)
primary: starfish.ImageStack = fov.get_image("primary")
dapi: starfish.ImageStack = fov.get_image("nuclei")

## Register images against DAPI stain

In [ ]:
from starfish.image import LearnTransform, ApplyTransform
learnTranslation: LearnTransform.Translation = LearnTransform.Translation(
    reference_stack=dapi,
    axes=Axes.ROUND,
    upsampling=1000
)
transforms: TransformsList = learnTranslation.run(fov)
ApplyTransform.warp(fov, transforms, in_place=True, verbose=True)

## Apply Filters
Applying a bandpass and Gaussian low pass filters interleaved with clip filters.

In [ ]:
from typing import Optional

from starfish import ImageStack
from starfish.image import Filter



def preprocess_fov(primary_fov_imagestack: ImageStack,
                  n_processes: Optional[int] = None,
                  is_volume: Optional[bool] = False) -> ImageStack:
   """Preprocess a Starfish field of view image stack in preparation for
   spot/pixel finding.

   NOTE: This preprocessing pipeline processes imagestacks in place!

   Args:
       primary_fov_imagestack (ImageStack): A starfish FOV Imagestack
       n_processes (Optional[int]): Number of processes to use for
           preprocessing steps. If None, uses the output of os.cpu_count().
           Defaults to None.

   Returns:
       ImageStack: A preprocessed starfish imagestack.
   """
   print("Applying First Clip...")
   first_clip = Filter.ClipPercentileToZero(p_min=75, p_max=100,
                                            is_volume=is_volume)
   first_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                  n_processes=n_processes)

   print("Applying Bandpass...")
   bpass = Filter.Bandpass(lshort=0.5, llong=7, threshold=1/(1<<16-1),
                           is_volume=is_volume)
   bpass.run(primary_fov_imagestack, in_place=True, verbose=True,
             n_processes=n_processes)

   print("Applying Second Clip...")
   second_clip = Filter.ClipValueToZero(v_min=1/(1<<16-1), is_volume=is_volume)
   second_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                   n_processes=n_processes)

   print("Applying Gaussian Low Pass...")
   z_gauss_filter = Filter.GaussianLowPass(sigma=(1, 0, 0), is_volume=True)
   z_gauss_filter.run(primary_fov_imagestack, in_place=True,
                      n_processes=n_processes)

   print("Applying Final Clips...")
   final_percent_clip = Filter.ClipPercentileToZero(p_min=90, min_coeff=1.75)
   final_percent_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                          n_processes=n_processes)

   final_value_clip = Filter.ClipValueToZero(v_max=1000/(1<<16-1))
   final_value_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                        n_processes=n_processes)

   return primary_fov_imagestack

primary = preprocess_fov(primary, n_processes=22)

## Find Spots
Use a local max peak finder with to automatically find the best threshold and find spots in the z-projected and filtered image. Each spot should decode to a gene since a spot in one channel corresponds to one gene.

In [ ]:
lmpf = starfish.spots.DetectSpots.LocalMaxPeakFinder(
min_distance=2,
stringency=0,
min_obj_area=4,
max_obj_area=600,
verbose=True,
is_volume=False)

intensities = lmpf.run(primary, n_processes=22)

## Decode spots
Next, spots are decoded.

In [ ]:
decoded = experiment.codebook.decode_per_round_max(intensities.fillna(0))
decode_mask = decoded['target'] != 'nan'